In [1]:
import requests
from bs4 import BeautifulSoup
import json
from collections import OrderedDict
from datetime import datetime
from __future__ import print_function 
import mysql.connector 
from mysql.connector import errorcode
import numpy_financial
from tabulate import tabulate
import time

C:\Users\dev46\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
def Updating_Yahoo_Table(Healthy_companies):
    counter = 0 
    for Ticker in Healthy_companies:
        counter = counter+1 

        try: 

            ### Storing Data from Yahoo's Data Analytics  
            Yahoo_data = YAHOO(Ticker)
            nextYR_growth = Yahoo_data[0]
            fiveYR_growth = Yahoo_data[1]
            Ticker = Ticker.replace('-','.')
            price = ThinkORSwim(Ticker)
            Current_Earnings = Yahoo_data[3]
            Future_Earnings = Yahoo_data[4]
            Rev_gr = Yahoo_data[5]
            try :
                Current_PE = round(price/Current_Earnings,2)
            except: 
                Current_PE = None
        except Exception as err: 
            print(err)


        try:
            cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
            cursor = cnx.cursor()

            query = ("Update yahoo_forecast "
                     "Set nextYR_gr = {}, "
                     " fiveYR_gr = {}, "
                     " close = {}, "
                     " Current_Earnings = {}, "
                     " Future_Earnings = {}, "
                     " Rev_gr = {}, "
                     " Current_PE = {} "
                     "Where Ticker = '{}'".format(nextYR_growth, fiveYR_growth, price, Current_Earnings, Future_Earnings, Rev_gr, Current_PE, Ticker))
            cursor.execute(query)   
            cnx.commit()
        except mysql.connector.ProgrammingError as err:
            x=1

        ### Clearing data, this is so values dont overlap in Yahoo's table
        YahooMySQL = [nextYR_growth, fiveYR_growth, price , Current_Earnings, Future_Earnings, Rev_gr, Current_PE]
        for items in YahooMySQL:
            items = None

        perc = round(((counter)/len(Healthy_companies))*100,2)
        string = """We have {} companies remaining! We are {}% done!                                      """.format(len(Healthy_companies)-counter, perc) 
        print(string, end='\r')



In [ ]:
def yahoo_forecastTable(Ticker):
    cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
    cursor = cnx.cursor()

    query = ("Select fiveYR_gr, close, Current_Earnings, Future_Earnings, Current_PE, Rev_gr, sic "
             "From yahoo_forecast "
             "inner join company_info on company_info.Ticker = yahoo_forecast.Ticker "
             "where yahoo_forecast.Ticker = '{}'"
            ).format(Ticker)

    cursor.execute(query)
    for (fiveYR_gr, close, Current_Earnings, Future_Earnings, Current_PE, Rev_gr, sic) in cursor:
        fiveYR_gr = float(fiveYR_gr)
        close = float(close)
        Current_Earnings = float(Current_Earnings)
        Future_Earnings = float(Future_Earnings)
        Current_PE = float(Current_PE)
        Rev_gr = float(Rev_gr)
        sic = sic
    return (fiveYR_gr, close, Current_Earnings, Future_Earnings, Current_PE, Rev_gr, sic)

In [ ]:
cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
cursor = cnx.cursor()

query = ("Select avg(yahoo_forecast.Current_PE) as avg_PE, company_info.sic "
         "from yahoo_forecast "
         "inner join company_info on company_info.CIK = yahoo_forecast.CIK "
         "group by company_info.sic")

cursor.execute(query)
sic_grouping = {}
for (avg_PE, sic) in cursor:
    sic_grouping[sic] = {}
    try: 
        sic_grouping[sic]['Avg_PE'] = float(avg_PE)
    except:
        sic_grouping[sic]['Avg_PE'] = None

In [ ]:
# Ticker = 'AMD'
def basic_overview(Ticker):
    cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
    cursor = cnx.cursor()
    query = ("Select TableRef, Ticker, form, val, frame "
             "From calctable "
             "Where  (form = '5YoYAvg' and TableRef = 'revenue' and frame Like 'TTM%' and Ticker = '{}' ) "
             "or (form = '3YoYAvg' and TableRef = 'revenue' and frame Like 'TTM%' and Ticker = '{}' ) "
             "or (form = '1YoYAvg' and TableRef = 'revenue' and frame Like 'TTM%' and Ticker = '{}' ) "
             "or (form = '5YoYAvg' and TableRef = 'total_debt' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '3YoYAvg' and TableRef = 'total_debt' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '1YoYAvg' and TableRef = 'total_debt' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '5YoYAvg' and TableRef = 'cash' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '3YoYAvg' and TableRef = 'cash' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '1YoYAvg' and TableRef = 'cash' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '5YoYAvg' and TableRef = 'total_debt_to_equity' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '3YoYAvg' and TableRef = 'total_debt_to_equity' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '1YoYAvg' and TableRef = 'total_debt_to_equity' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '5YoYAvg' and TableRef = 'div_per_share' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '3YoYAvg' and TableRef = 'div_per_share' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '1YoYAvg' and TableRef = 'div_per_share' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '5YoYAvg' and TableRef = 'earnings_per_share' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '3YoYAvg' and TableRef = 'earnings_per_share' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '1YoYAvg' and TableRef = 'earnings_per_share' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '5YoYAvg' and TableRef = 'levered_free_cash_flow' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '3YoYAvg' and TableRef = 'levered_free_cash_flow' and frame Like 'CY%' and Ticker = '{}' ) "
             "or (form = '1YoYAvg' and TableRef = 'levered_free_cash_flow' and frame Like 'CY%' and Ticker = '{}' ) ".format( Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker, Ticker))
    cursor.execute(query)
    Ticker_Dictionary = {}

    for (TableRef, Ticker, form, val, frame) in cursor:
        try:
            if Ticker_Dictionary[TableRef] != {}:
                Ticker_Dictionary[TableRef][form] = float(val)
        except: 
            Ticker_Dictionary[TableRef] = {}
            Ticker_Dictionary[TableRef][form] = float(val)                 

    for Table in Ticker_Dictionary:
        # Colletcing the 5 past year values CY
        cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
        cursor = cnx.cursor()
        query = ("Select Num_From_Current_Date, val "
                 "From {} "
                 "Where (Ticker = '{}' and Num_From_Current_Date = 0 and frame like 'CY%') "
                 "or (Ticker = '{}' and Num_From_Current_Date = 1 and frame like 'CY%') "
                 "or (Ticker = '{}' and Num_From_Current_Date = 2 and frame like 'CY%') "
                 "or (Ticker = '{}' and Num_From_Current_Date = 3 and frame like 'CY%') "
                 "or (Ticker = '{}' and Num_From_Current_Date = 4 and frame like 'CY%')".format(Table, Ticker, Ticker, Ticker, Ticker, Ticker))
        cursor.execute(query)
        for (Num_From_Current_Date, val) in cursor:
            Value = float(val)
            if Value >= 1000000000:
                Value = round(Value/1000000000,2)
                Value = '{} bil'.format(Value)
            else:
                if Value >= 1000000:
                    Value = round(Value/1000000,2)
                    Value = '{} mil'.format(Value)
                else:
                    if Value >= 1000:
                        Value = round(Value/1000,2)
                        Value = '{} grand'.format(Value)
            Ticker_Dictionary[Table][Num_From_Current_Date] = Value

        # Overwriting the CY if TTM is present
        cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
        cursor = cnx.cursor()
        query = ("Select Num_From_Current_Date, val "
                 "From {} "
                 "Where (Ticker = '{}' and Num_From_Current_Date = 0 and frame like 'TTM%') "
                 "or (Ticker = '{}' and Num_From_Current_Date = 1 and frame like 'TTM%') "
                 "or (Ticker = '{}' and Num_From_Current_Date = 2 and frame like 'TTM%') "
                 "or (Ticker = '{}' and Num_From_Current_Date = 3 and frame like 'TTM%') "
                 "or (Ticker = '{}' and Num_From_Current_Date = 4 and frame like 'TTM%')".format(Table, Ticker, Ticker, Ticker, Ticker, Ticker))
        cursor.execute(query)
        for (Num_From_Current_Date, val) in cursor:
            Value = float(val)
            if Value >= 1000000000:
                Value = round(Value/1000000000,2)
                Value = '{} bil'.format(Value)
            else:
                if Value >= 1000000:
                    Value = round(Value/1000000,2)
                    Value = '{} mil'.format(Value)
                else:
                    if Value >= 1000:
                        Value = round(Value/1000,2)
                        Value = '{} grand'.format(Value)
            Ticker_Dictionary[Table][Num_From_Current_Date] = Value

    CheckList = ['0', '1', '2', '3', '4', '1YoYAvg', '3YoYAvg', '5YoYAvg']
    for Table in Ticker_Dictionary: 
        for items in CheckList:
            try:
                # This should trigger an error 
                if Ticker_Dictionary[Table][items] == 'N/A':
                    Ticker_Dictionary[Table][items] = 'N/A'
            except:
                Ticker_Dictionary[Table][items] = 'N/A'

    Header = ['4yrs', '3yrs', '2yrs', '1yrs', 'Current', '1YoYAvg', '3YoYAvg', '5YoYAvg']
    Data = []
    for Table in Ticker_Dictionary:
        Data.append([Table, Ticker_Dictionary[Table]['4'], Ticker_Dictionary[Table]['3'], 
              Ticker_Dictionary[Table]['2'], Ticker_Dictionary[Table]['1'], Ticker_Dictionary[Table]['0'], 
              Ticker_Dictionary[Table]['1YoYAvg'], Ticker_Dictionary[Table]['3YoYAvg'], Ticker_Dictionary[Table]['5YoYAvg']])

    print(tabulate(Data, headers = Header))

In [ ]:
    def Forecasting_info(Ticker):
        cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
        cursor = cnx.cursor()

        Header = ['Ticker', 'fiveYR_gr', 'nextYR_gr', 'Current_Earnings', 'Future_Earnings', 'Rev_gr', 'Current_PE']

        query = ("select company_info.Ticker as Ticker, fiveYR_gr, nextYR_gr, Current_Earnings, Future_Earnings, Rev_gr, Current_PE "
                 "from company_info "
                 "inner join yahoo_forecast on yahoo_forecast.CIK = company_info.CIK "
                 "where company_info.Ticker = '{}' "
                 "order by fiveYR_gr desc".format(Ticker)) 
        cursor.execute(query)
        for (Ticker, fiveYR_gr, nextYR_gr, Current_Earnings, Future_Earnings, Rev_gr, Current_PE) in cursor:
            try: 
                fiveYR_gr = float(fiveYR_gr)
            except:
                fiveYR_gr = 'N/A'
            try: 
                nextYR_gr = float(nextYR_gr)
            except:
                nextYR_gr = 'N/A'
            try: 
                Current_Earnings = float(Current_Earnings)
            except:
                Current_Earnings = 'N/A'
            try: 
                Future_Earnings = float(Future_Earnings)
            except:
                Future_Earnings = 'N/A'      
            try: 
                Rev_gr = float(Rev_gr)
            except:
                Rev_gr = 'N/A'    
            try: 
                Current_PE = float(Current_PE)
            except:
                Current_PE = 'N/A'       
            Data = [Ticker, fiveYR_gr, nextYR_gr, Current_Earnings, Future_Earnings, Rev_gr, 
                    Current_PE]
        print(tabulate([Data], headers = Header))
